<a href="https://colab.research.google.com/github/vxima/CVMI/blob/main/Task_Dense_(Shuffled).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####INCLUDES
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import imageio
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
##clonar repositorio
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

fatal: destination path 'covid-chestxray-dataset' already exists and is not an empty directory.


In [ ]:
##DATASET
class CovidXraysDataset(Dataset):
    """Covid ChestXrays."""

    def __init__(self, csv_file, image_dir, transform=None):
        
        self.metadata = pd.read_csv(csv_file)
        self.image_dir = image_dir

    def __len__ (self):
        return len(self.metadata.filename)

    def __getitem__(self, idx):
        image_pathing = self.metadata.filename[idx]
        image_path = str(self.image_dir) + '/' + str(image_pathing)
        preprocess = transforms.Compose([
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),

        ])
        img = Image.open(image_path)
        image = preprocess(img)
        data = self.metadata.iloc[idx]
        if data[4] == "Pneumonia/Viral/COVID-19":
          sample = { 'label': torch.Tensor([0]).float() , 'image': image }
        else:
          sample = {'label': torch.Tensor([1]).float() , 'image': image }
        
        return sample
    def __getItemShuffle__(self , idx):
        image_pathing = self.metadata.filename[idx]
        image_path = str(self.image_dir) + '/' + str(image_pathing)
        preprocess = transforms.Compose([
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),

        ])
        img = Image.open(image_path)
        image = preprocess(img)
        data = self.metadata.iloc[idx]
        sample = {'label': data[1], 'image': image }
        return sample

In [ ]:
##DATASET SHUFFLED
def Data2Shuffle():

  files = CovidXraysDataset(csv_file='/content/gdrive/MyDrive/Colab Notebooks/metadata_shuffle.csv', image_dir='/content/covid-chestxray-dataset/images')
  l = len(files)
  filename = []
  for i in range(0 , l):
    
    try:
        aux_image = files.__getItemShuffle__(i)['image']
        aux_label = files.__getItemShuffle__(i)['label']
        if aux_label == 'tensor([0, 1])':
          aux_label = torch.Tensor([0 , 1]).float()
        else:
          aux_label = torch.Tensor([1 , 0]).float()
        aux_lst = (aux_image , aux_label)
        if aux_image.shape[0] == 3:
          filename.append(aux_lst)
    except FileNotFoundError:
        pass
  print("Tamanho do dataset redzido para: " , len(filename))
  return filename
listaShuffled = Data2Shuffle()

Tamanho do dataset redzido para:  478


In [ ]:
##SPLIT DATASET

#train_filename = listaShuffled[2:828]
#dev_filename = listaShuffled[829:879]
#test_filename = listaShuffled[880:927]

l = len(listaShuffled) #atualizando o tamanho de l
split1 = int(0.8*l)
split2 = int(0.9*l)
train_filename = listaShuffled[0:split1]
dev_filename = listaShuffled[split1:split2]
test_filename = listaShuffled[split2:]
PATH_SHUFFLE = "/content/gdrive/MyDrive/savepoint_shuffle.pt"

In [ ]:
##DENSENET
device = torch.device("cuda:0")
dense = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=False, )
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet169', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet201', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet161', pretrained=True)
dense.classifier = torch.nn.Linear(1024,2)
dense.to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
##HIPER PARAMETROS
Batch_Size = 16
Epochs = 128
Learning_Rate = 0.00001

In [ ]:
d##TREINAMENTO
def train(dataset,model,criterion,optimizer,batch_size ,total_epochs, device=torch.device('cpu')):
  train_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size)
  model.train()
  for epoch in range(total_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(torch.nn.functional.softmax(output), target.to(device).float())
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
  return model

In [ ]:
#Loss function
criterion = torch.nn.BCELoss()
#Optimizer
optimizer = torch.optim.Adam(dense.parameters(),lr=Learning_Rate)

In [ ]:
##TRAIN
dense = train(train_filename , dense , criterion , optimizer , Batch_Size , Epochs, device=device)


Train Epoch: 0 [0/382 (0%)]	Loss: 0.000046
Train Epoch: 0 [160/382 (42%)]	Loss: 0.000247
Train Epoch: 0 [320/382 (83%)]	Loss: 0.002634
Train Epoch: 1 [0/382 (0%)]	Loss: 0.000022
Train Epoch: 1 [160/382 (42%)]	Loss: 0.000122
Train Epoch: 1 [320/382 (83%)]	Loss: 0.001441
Train Epoch: 2 [0/382 (0%)]	Loss: 0.000029
Train Epoch: 2 [160/382 (42%)]	Loss: 0.000077
Train Epoch: 2 [320/382 (83%)]	Loss: 0.001062
Train Epoch: 3 [0/382 (0%)]	Loss: 0.000033
Train Epoch: 3 [160/382 (42%)]	Loss: 0.000075
Train Epoch: 3 [320/382 (83%)]	Loss: 0.001047
Train Epoch: 4 [0/382 (0%)]	Loss: 0.000033
Train Epoch: 4 [160/382 (42%)]	Loss: 0.000077
Train Epoch: 4 [320/382 (83%)]	Loss: 0.001059
Train Epoch: 5 [0/382 (0%)]	Loss: 0.000032
Train Epoch: 5 [160/382 (42%)]	Loss: 0.000079
Train Epoch: 5 [320/382 (83%)]	Loss: 0.001072
Train Epoch: 6 [0/382 (0%)]	Loss: 0.000031
Train Epoch: 6 [160/382 (42%)]	Loss: 0.000081
Train Epoch: 6 [320/382 (83%)]	Loss: 0.001084
Train Epoch: 7 [0/382 (0%)]	Loss: 0.000031
Train Epoch:

In [ ]:
##SAVE
torch.save(dense , PATH_SHUFFLE)
print(dense)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
##LOAD
tst = torch.load(PATH_SHUFFLE)
tst.eval()


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
##PREDICAO
def Validation(dataset, model, batch_size=1, device=torch.device('cpu')):
    model.eval()
    with torch.no_grad():
      Y_pred = np.array((0,))
      Y_true = []
      dev_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
      for batch_idx , (data, target) in enumerate(dev_loader):
        
        pred = model(data.to(device))
        pred = torch.nn.functional.softmax(pred , 1)
        pred = torch.argmax(pred , 1).detach().cpu().numpy()
        true = target.detach().cpu().numpy()
        Y_pred = np.concatenate((Y_pred , pred))
      #agora fazer a lista de Y_true
        if torch.equal(target[0] ,torch.tensor([0. , 1.])):
          Y_true.append(1)
        else:
          Y_true.append(0)
      return Y_pred[1:] , Y_true





In [ ]:
##METRICAS

class Metrics:
    def __init__(self  , Y_true , Y_pred):
      self.y_true = Y_true
      self.y_pred = Y_pred
    def metric_f1_score(self):
      return f1_score(self.y_true, self.y_pred, average='macro')
    def metric_confusion_matrix(self):
      return confusion_matrix(self.y_true, self.y_pred)
    def metric_balanced_accuracy_score(self):
      return balanced_accuracy_score(self.y_true, self.y_pred)
    def metric_matthews_corrcoef(self):
      return matthews_corrcoef(self.y_true, self.y_pred)



In [ ]:
#Testando as metricas
Y_pred , Y_true = Validation(dev_filename, tst, device=device)
testador = Metrics(Y_true, Y_pred)
print("F1_score: " ,testador.metric_f1_score())
print("Confusion_matrix: " ,testador.metric_confusion_matrix())
print("Balanced_accuracy_score: " , testador.metric_balanced_accuracy_score())
print("Matthews_corrcoef :" , testador.metric_matthews_corrcoef())

F1_score:  0.6647619047619048
Confusion_matrix:  [[ 5  5]
 [ 6 32]]
Balanced_accuracy_score:  0.6710526315789473
Matthews_corrcoef : 0.330563082296276
